In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas


import csv
import os

import numpy as np

from dataloader import image_dataloader
from model import swin_t
from train import train_model

from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
import torch.optim as optim

In [ ]:
train_label_path = 'train.csv'
val_label_path = 'val.csv'

train_image_path = '.../train/rgb/'
val_image_path = '.../val/rgb/'

train_VSI_path = '.../train/vsi/'
val_VSI_path = '.../val/vsi/'

train_TDA_path = '.../tda/vsi/'
val_TDA_path = '.../tda/vsi/'

In [ ]:
image_datasets, dataloaders = image_dataloader(train_image_path, train_VSI_path, train_TDA_path, train_label_path = 'train.csv', 
                   bs = 5, 
                   val_image_path = val_image_path , val_VSI_path = val_VSI_path, val_TDA_path = val_TDA_path, val_label_path = 'val.csv' )

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model = swin_t()
model.to(device)

In [ ]:

criterion = nn.BCEWithLogitsLoss()
lr = 1e-5
optimizer = optim.Adam(model.parameters(), lr=1e-5)
num_epochs = 30
save_path = '.../result/model/'

##### Adaptive lr #####
#gamma = 0.7
#scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=30, device, image_datasets, dataloaders, save_path)
(model1, train_f1, train_accuracy, val_f1, val_accuracy, save_outputs, save_labels, save_preds) = model_trained

Save actual labels and predictions for ROC curve plot

In [ ]:
save_labels = [int(labels) for labels in save_labels]
save_preds = [int(preds) for preds in save_preds]

output_document = []
for i in range(len(val_image_path)):
    single_list = []
    single_list.append(save_labels[i])
    single_list.append(save_outputs[i])
    single_list.append(save_preds[i])
    output_document.append(single_list)

# field names 
fields = ['Actual Label', 'Output', 'Prediction'] 
    
# data rows of csv file 
rows = output_document
  
with open('/result/outputs/best_hybrid', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)

print(pd.read_csv('task01_best_RGB'))